## Préliminaires - prise de connaissance des données

### Importations des modules et des données

In [167]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
import re
#import scipy.stats as st
#import statsmodels.api as sm
#from sklearn.datasets import load_iris
#iris_df_ori = load_iris()
####     A ESSAYER      #######################################################
# GRAPHES INTERACTIFS
# %matplotlib notebook # ne pas oublier de désactiver l'interctivité (bouton topright)

# POUR LES GRAPHIQUES
# % matplotlib inline 
# plt.rcParams['figure.figsize'] = [9.5, 6] # ajuster la taille

# POUR DESACTIVER LA TOOLBOX GRAPHES TOP GRANDS
# %%javascript
# IPython.OutputArea.prototype._should_scroll = function(lines) {
#     return false;
# }


L'ensemble des données téléchargées se compose de 5 fichiers .csv et d'un fichier excel comportant 5 onglets.
Il semble que l'intégralité des données des fichiers .csv soit reprise dans chacun des onglets du fichier Excel.

Dans ce notebook, on appellera "base de donnée" l'ensemble des données, et "table" chacun des onglets ou fichier .csv correspondant.

In [130]:
# Utilisé la fonction dropna (colonne nulle) pour éliminer les colonnes fantômes "Unamed en fin de tableau"
data = pd.read_csv("../DONNEES/EdStatsData.csv").dropna(how='all', axis='columns')
country = pd.read_csv("../DONNEES/EdStatsCountry.csv").dropna(how='all', axis='columns')
cnt_ser = pd.read_csv("../DONNEES/EdStatsCountry-Series.csv").dropna(how='all', axis='columns')
series = pd.read_csv("../DONNEES/EdStatsSeries.csv").dropna(how='all', axis='columns')
footnote = pd.read_csv("../DONNEES/EdStatsFootNote.csv").dropna(how='all', axis='columns')

### Exploration du contenu des 5 tables

In [119]:
## Fonction qui extrait les données principales relatives aux colonnes

def infos_df (df):
    infos = pd.DataFrame(df.dtypes).T.rename(index={0:'Type'}) 
    infos = infos.append(pd.DataFrame([df[col].size for col in df.columns],index=df.columns)\
                               .T.rename(index={0:'Nbe lignes'}))
    infos = infos.append(pd.DataFrame(df.isna().sum()).T.rename(index={0:'Nbe NaN'}))
    infos = infos.append(pd.DataFrame(infos.loc["Nbe lignes"]-infos.loc["Nbe NaN"])\
                         .T.rename(index = {0:"Nbe Entrées"}))
    infos = infos.append(pd.DataFrame(df.isna().sum()/df.shape[0]*100).T.rename(index={0:'% NaN'}))
    infos = infos.append(pd.DataFrame([df[col].unique().size for col in df.columns],index=df.columns)\
                               .T.rename(index={0:'Nbe uniques'}))
    return infos

In [145]:
###############
#     Data    #
###############
infos_df(data)

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
Type,object,object,object,object,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Nbe lignes,886930,886930,886930,886930,886930,886930,886930,886930,886930,886930,...,886930,886930,886930,886930,886930,886930,886930,886930,886930,886930
Nbe NaN,0,0,0,0,814642,851393,851311,851385,851200,799624,...,835494,835494,835494,835494,835494,835494,835494,835494,835494,835494
Nbe Entrées,886930,886930,886930,886930,72288,35537,35619,35545,35730,87306,...,51436,51436,51436,51436,51436,51436,51436,51436,51436,51436
% NaN,0,0,0,0,91.8496,95.9933,95.984,95.9924,95.9715,90.1564,...,94.2007,94.2007,94.2007,94.2007,94.2007,94.2007,94.2007,94.2007,94.2007,94.2007
Nbe uniques,242,242,3665,3665,24596,30893,30983,30989,31140,37839,...,8028,7915,7801,7701,7563,7467,7336,7151,7045,6915


In [146]:
###############
#   Country   #
###############
infos_df(country)

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
Type,object,object,object,object,object,object,object,object,object,object,...,object,object,object,object,object,object,object,float64,float64,object
Nbe lignes,241,241,241,241,241,241,241,241,241,241,...,241,241,241,241,241,241,241,241,241,241
Nbe NaN,0,0,0,0,3,26,96,27,27,1,...,80,60,28,100,81,130,99,134,56,62
Nbe Entrées,241,241,241,241,238,215,145,214,214,240,...,161,181,213,141,160,111,142,107,185,179
% NaN,0,0,0,0,1.24481,10.7884,39.834,11.2033,11.2033,0.414938,...,33.195,24.8963,11.6183,41.4938,33.61,53.9419,41.0788,55.6017,23.2365,25.7261
Nbe uniques,241,241,241,241,239,153,132,8,6,241,...,3,3,28,61,76,3,36,12,14,21


In [147]:
#################
#     Series    #
#################
infos_df(series)

,Series Code,Topic,Indicator Name,Short definition,Long definition,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,General comments,Source,Statistical concept and methodology,Development relevance,Related source links
Type,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
Nbe lignes,3665,3665,3665,3665,3665,3665,3665,3665,3665,3665,3665,3665,3665,3665,3665
Nbe NaN,0,0,0,1509,0,3566,3351,3113,3618,3651,3651,0,3642,3662,3450
Nbe Entrées,3665,3665,3665,2156,3665,99,314,552,47,14,14,3665,23,3,215
% NaN,0,0,0,41.1733,0,97.2988,91.4325,84.9386,98.7176,99.618,99.618,0,99.3724,99.9181,94.1337
Nbe uniques,3665,37,3665,1170,2060,2,5,15,4,10,9,31,3,2,2


In [148]:
#########################
#     Country-Series    #
#########################
infos_df(cnt_ser)

,CountryCode,SeriesCode,DESCRIPTION
Type,object,object,object
Nbe lignes,613,613,613
Nbe NaN,0,0,0
Nbe Entrées,613,613,613
% NaN,0,0,0
Nbe uniques,211,21,97


In [149]:
#################
#    FootNote   #
#################
infos_df(footnote)

,CountryCode,SeriesCode,Year,DESCRIPTION
Type,object,object,object,object
Nbe lignes,643638,643638,643638,643638
Nbe NaN,0,0,0,0
Nbe Entrées,643638,643638,643638,643638
% NaN,0,0,0,0
Nbe uniques,239,1558,56,9102


### Vérification des données contenues dans les tables

#### Noms et codes des pays

Les tables "Data" et "Country" n'ont pas le même nombre d'entrée uniques pour la colonne "Country Code" :
la table "Country" contient un pays en moins.

On cherche à déterminer ce pays :

In [429]:
# Fonction qui trouve les éléments différents dans deux tableaux (réciproque)

def Diff(tab1, tab2):  # le nombre total de différences
    tab_dif = [i for i in tab1 + tab2 if i not in tab1 or i not in tab2] 
    return tab_dif 

def Diff_bis(tab1, tab2): # compare en détaillant les entrées différentes de tab1, puis de tab2
    tab_dif = [i for i in tab1 + tab2 if i not in tab1 or i not in tab2] 
    return (set(tab1)-set(tab2),set(tab2)-set(tab1))

In [430]:
pays = Diff(list(data["Country Code"].unique()),list(country["Country Code"].unique()))
print("Code et nom du pays manquant dans Country : {},{}"\
  .format(pays,[data["Country Name"][data["Country Code"]==pays[i]].iloc[0] for i in range(len(pays))]))

Code et nom du pays manquant dans Country : ['VGB'],['British Virgin Islands']


Un "Country Code" correspond-il à un seul "Country Name" dans la table "Data" ?

In [431]:
# Fonction comparant la correspondance unique entre les valeurs d'une même ligne de deux colonnes d'une base
# (bijection entre les valeurs de col1 et de col2)

def Adeq (df, nom_col1,nom_col2):
    mon_zip = zip(df[nom_col1], df[nom_col2]) # associe les entrées des deux colonnes en tuples
    nbe_comb = len(set(mon_zip)) # retourne les valeurs uniques des tuples
    return nbe_comb==df[nom_col1].unique().size # si le nbe est le même que les valeurs uniques, c'est bon

# Version très lourde de la même fonction
# def Adeq_Col(df, nom_col1,nom_col2): # dans une même base df
#     tab = []
#     for i in range(len(df[nom_col1].unique())):
#         uni_i = df[nom_col1].unique()[i]
#         # liste des valeurs de col2 correspondant à la ième valeur unique de col1
#         li = df[df[nom_col1]==uni_i][nom_col2]
#         temp = 0
#         for j in range(len(li)): # comparaison de toutes les valeurs de li une à une
#             if li.iloc[j] != li.iloc[0]: # on prend la première valeur (0) comme référence
#                 temp+=1
#         tab.append(temp)
#     return temp # la valeur retournée est le nombre de valeurs différentes

In [432]:
Adeq(data, "Country Code", "Country Name")

True

In [423]:
a = [3,2,1,5,1,6,1]
b = ['c','b','a','e','a','u','a']
z = zip(a,b)
set(z)

{(1, 'a'), (2, 'b'), (3, 'c'), (5, 'e'), (6, 'u')}

Y a-t-il la même correspondance entre les codes pays ("Country Code") et les noms de pays dans les tables "Data" ("Country Name") et "Country" ("Table Name") ?
Si oui, on ne gardera qu'une des deux colonnes.

In [ ]:
code_name_data = 

In [379]:
li = data[data["Country Code"]==cnt_code_uni[10]]["Country Name"]
temp = 0
for j in range(len(li)):
    if li.iloc[j] != li.iloc[0]:
        temp+=1
tab.append(temp)
temp

0

In [376]:
li.iloc[0]

'Latin America & Caribbean (excluding high income)'

#### Noms et codes des indicateurs statistiques

On remarque que malgré les noms différents, les colonnes "Indicator Code" de la table "Data" et "Series Code" de la table "Series" semblent faire référence au même code d'indicateur statistique, et ont, elles aussi, le même nombre d'entrées uniques de 3665.
Regardons si les entrées uniques de ces deux colonnes correspondent :

In [315]:
code = Diff_bis(list(data["Indicator Code"].unique()),list(series["Series Code"].unique()))
len(code[0]),len(code[1])

(53, 53)

53 entrées uniques de la colonne "Indicator Code" de la table "Data" ne sont pas dans la table "Series" et que le même nombre 53 d'entrées uniques de la colonne "Series Code" ne sont pas dans la table "Data".

In [329]:
list(code[0])[:2], list(code[1])[:2]

(['UIS.AIR.1.GLAST.GPI', 'UIS.NIR.1.AGP1.M'],
 ['UIS.NIR.1.AgM1.F', 'UIS.E.23.Pu'])

On constate que tous les codes différents de la colonne "Series Code" (table "Series") ont certaines lettres en minuscule.
Essayons donc de mettre toutes les entrées en majuscule et de refaire la comparaison.

In [346]:
# pour la table "Series"
test_series = pd.Series([series["Series Code"][i].upper() for i in range(series["Series Code"].index.size)],\
                 index = series["Series Code"].index)
# pour la table "Data"
test_data = pd.Series([data["Indicator Code"][i].upper() for i in range(data["Indicator Code"].index.size)],\
                 index = data["Indicator Code"].index)

code = Diff_bis(list(test_data.unique()),list(test_series.unique()))
len(code[0]),len(code[1])

(3, 3)

In [347]:
list(code[0]), list(code[1])

(['SE.SEC.DURS.LO', 'UIS.OE.56.40510', 'SE.SEC.ENRR.UP.FE'],
 ['SE.SEC.DURS.LO ', 'UIS.OE.56.40510 ', 'SE.SEC.ENRR.UP.FE '])

La liste ci-dessus montre qu'il faudra également retirer les espaces de la colonne "Series Code" de la table "Series".

In [350]:
test_series = pd.Series([test_series.iloc[i].replace(" ", "") for i in range(test_series.index.size)],\
                 index = series["Series Code"].index)

In [351]:
code = Diff_bis(list(test_data.unique()),list(test_series.unique()))
len(code[0]),len(code[1])

(0, 0)

In [ ]:
Il est

On remarque par ailleurs que les colonnes "Indicator Name" des tables "Data" et "Series" ont le même nombre d'entrées uniques (3665). Vérifions si ces entrées uniques correspondent :

In [262]:
indic = Diff_bis(list(data["Indicator Name"].unique()),list(series["Indicator Name"].unique()))
len(indic[0]), len(indic[1])

(462, 462)

On remarque que 462 entrées uniques de la colonne "Indicator Name" de la table "data" ne sont pas dans la table "series" et que le même nombre 462 d'entrées uniques de la même colonne de la table "series" ne sont pas dans la table "data".

Cela pourrait-il indiquer qu'il s'agirait des mêmes entrées qui ne présentent que de petites variations d'écriture, comme c'est le cas pour les deux entrées ci-dessous ?

In [ ]:
indic = Diff_bis(list(data["Indicator Code"].unique()),list(series["Series Code"].unique()))
len(indic[0]), len(indic[1])

In [299]:
list(indic[0])[7],list(indic[1])[8]

('SABER: (School Autonomy Accountability) Policy Goal 2: Level of autonomy in personnel management',
 'SABER: (School Autonomy and Accountability) Policy Goal 2: Level of autonomy in personnel management')

In [314]:
a = data["Indicator Code"][data["Indicator Name"]=='SABER: (School Autonomy Accountability) Policy Goal 2: Level of autonomy in personnel management'].iloc[0:3]
b = series["Series Code"][series["Indicator Name"]=='SABER: (School Autonomy and Accountability) Policy Goal 2: Level of autonomy in personnel management']
a,b

(2734     SABER.SCH.ATNM.GOAL2
 6399     SABER.SCH.ATNM.GOAL2
 10064    SABER.SCH.ATNM.GOAL2
 Name: Indicator Code, dtype: object, 2095    SABER.SCH.ATNM.GOAL2
 Name: Series Code, dtype: object)

#### Colonnes de la table "CountrySeries" et "FootNote"

Les entrées des colonnes "CountryCode" et "SeriesCode" des tables "CountrySeries" et "FootNote" existent-ils dans la liste de référence "Country Code" de la table "Data" ?

Les années de la table "FootNote" sont-elles dans les années correspondant aux noms de colonnes de la table "Data" ?

In [285]:
# liste des années de la colonne "Year" de la table "FootNote"
#set(footnote["Year"].unique())
footnote[footnote["Year"]=='yr2012']

,CountryCode,SeriesCode,Year,DESCRIPTION
150668,EAP,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
157898,ECA,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
172364,EMU,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
232331,HIC,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
308605,LAC,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
344095,LIC,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
354902,LMC,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
360606,LMY,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
404072,MIC,SP.POP.GROW,yr2012,5-year average value between 0s and 5s
416009,MNA,SP.POP.GROW,yr2012,5-year average value between 0s and 5s


In [197]:
tab1 = list(['a', 'b', 'c', 'd', 'e'])
tab2 = list(['adfc', 'c', 'd', 'e'])

list1 = [10, 15, 20, 45, 30, 35, 25]
list2 = [25, 40, 35] 


def Diff(tab1, tab2): 
    tab_dif = [i for i in tab1 + tab2 if i not in tab1 or i not in tab2] 
    return tab_dif 

Diff(list1, list2)
(set(tab2)-set(tab1)),(set(tab1)-set(tab2))


({'adfc'}, {'a', 'b'})

In [150]:
# extrait, transpose et renomme la ligne
infos_df = pd.DataFrame(data.dtypes).T.rename(index={0:'Type'}) 
infos_df = infos_df.append(pd.DataFrame(data.isna().sum()).T.rename(index={0:'Nbe NaN'}))
infos_df = infos_df.append(pd.DataFrame(data.isna().sum()/data.shape[0]*100).T.rename(index={0:'% NaN'}))
infos_df = infos_df.append(pd.DataFrame([data[col].unique().size for col in data.columns],index=data.columns)\
                           .T.rename(index={0:'Nbe uniques'}))
infos_df = infos_df.append(pd.DataFrame([data[col].size for col in data.columns],index=data.columns)\
                           .T.rename(index={0:'Nbe lignes'}))

In [166]:
tab = ['a', 'b', 'c', 'd', 'e']
[tab[i+1] for i in range(5) if tab[i]=='d'][0]

'e'

In [121]:
############DATA
# colonnes
print("L'onglet Data a {} lignes et {} colonnes, qui sont : ".format(data.index.size, data.columns.size))
print("{} : ".format(data.columns))
# nbe valeurs uniques 
print("Les 4 premières colonnes qui sont {} : "\
      .format(list(data.columns[:4])))
print("prennent respectivement {} valeurs uniques."\
      .format([data[col].unique().size for col in data.columns[:4]]))
print("---------------------------------------------")
#########COUNTRY
# colonnes
print("L'onglet Country a {} lignes et {} colonnes, qui sont : {}"\
      .format(country.index.size, country.columns.size, list(country.columns)))
# nbe valeurs uniques 
print("prennent respectivement {} valeurs uniques."\
      .format([country[col].unique().size for col in country.columns]))
print("---------------------------------------------")
########SERIES
# colonnes
print("L'onglet Series a {} lignes et {} colonnes, qui sont : {}"\
      .format(series.index.size, series.columns.size, list(series.columns)))
# nbe valeurs uniques 
print("prennent respectivement {} valeurs uniques."\
      .format([series[col].unique().size for col in series.columns]))
print("---------------------------------------------")
###########COUNTRY-SERIES
# colonnes
print("L'onglet Country-Series a {} lignes et {} colonnes, qui sont : {}"\
      .format(cnt_ser.index.size, cnt_ser.columns.size, list(cnt_ser.columns)))
# nbe valeurs uniques 
print("prennent respectivement {} valeurs uniques."\
      .format([cnt_ser[col].unique().size for col in cnt_ser.columns]))
print("---------------------------------------------")
###########FOOTNOTE
# colonnes
print("L'onglet Series a {} lignes et {} colonnes, qui sont : {}"\
      .format(footnote.index.size, footnote.columns.size, list(footnote.columns)))
# nbe valeurs uniques 
print("prennent respectivement {} valeurs uniques."\
      .format([footnote[col].unique().size for col in footnote.columns]))

L'onglet Data a 886930 lignes et 70 colonnes, qui sont : 
Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2020', '2025', '2030', '2035', '2040', '2045',
       '2050', '2055', '2060', '2065', '2070', '2075', '2080', '2085', '2090',
       '2095', '2100', 'Unnamed: 69'],
      dtype='object') : 
Les 4 premières colonnes qui sont ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] : 
prennent respectivement [242, 242, 3665, 3665] valeurs uniques.
---------------------------------------------
L'onglet Country a 241 lignes et 3

In [151]:
df = pd.DataFrame([0,1,2,3,4], index = ['a', 'b', 'Unnamed', 'd', 'e'])
df = df.T
~df.columns.str.match('Unnamed')
df.loc[:, [True, False, True]]

,a,Unnamed
0,0,2
